In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("harishkumardatalab/food-image-classification-dataset")

print("Path to dataset files:", path)

100%|██████████████████████████████████████| 1.68G/1.68G [01:34<00:00, 19.1MB/s]

Extracting files...


Path to dataset files: /Users/ashimaarora/.cache/kagglehub/datasets/harishkumardatalab/food-image-classification-dataset/versions/1
